In [2]:
! pip install pylatexenc
! pip install transformers                               
# sentencepiece library for t5 transformers
! pip install sentencepiece
! pip install sumy

     |████████████████████████████████| 162 kB 5.1 MB/s 
  Created wheel for pylatexenc: filename=pylatexenc-2.10-py3-none-any.whl size=136835 sha256=8318fda17b6aad1acdc9e1e3f2b41ad2fe0c8a851e3edc3c76fb51a30caec4c6
  Stored in directory: /root/.cache/pip/wheels/f1/8a/f5/33ee79d4473eb201b519fa40f989b842e373237395a3421f52
Successfully built pylatexenc
     |████████████████████████████████| 3.5 MB 5.0 MB/s 
     |████████████████████████████████| 6.8 MB 36.2 MB/s 
     |████████████████████████████████| 596 kB 66.9 MB/s 
     |████████████████████████████████| 895 kB 61.4 MB/s 
     |████████████████████████████████| 67 kB 5.2 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 1.2 MB 5.3 MB/s 
     |████████████████████████████████| 87 kB 3.5 MB/s 
     |████████████████████████████████| 10.1 MB 32.3 MB/s 
  Created wheel for breadability: filena

In [7]:
from nltk.tokenize import word_tokenize
text = "God is Great! I won a lottery."
print(word_tokenize(text))



['God', 'is', 'Great', '!', 'I', 'won', 'a', 'lottery', '.']


In [9]:
number_of_words = len((word_tokenize(text)))

In [10]:
number_of_words

9

In [11]:
len(text.split())

7

In [12]:
text.split()

['God', 'is', 'Great!', 'I', 'won', 'a', 'lottery.']

In [14]:
from nltk.tokenize import sent_tokenize
text = "God is Great! I won a lottery."
print(sent_tokenize(text))


['God is Great!', 'I won a lottery.']


In [58]:
import math
def set_summary_length(text):
  number_of_words = len(word_tokenize(text))
  number_of_sentences = len(sent_tokenize(text))
  print(number_of_words)
  print(number_of_sentences)
  summary_senteces_len = math.ceil(number_of_sentences / 3)
  summary_min_tokens = math.ceil(number_of_words / 4) 
  summary_max_tokens = number_of_words // 3
  return summary_min_tokens, summary_max_tokens,summary_senteces_len

In [59]:
len(sent_tokenize(text))

2

In [43]:
ai_blog_wiki = """
Artificial intelligence (AI) is intelligence demonstrated by machines, as opposed to the natural intelligence displayed by humans or animals. Leading AI textbooks define the field as the study of "intelligent agents": any system that perceives its environment and takes actions that maximize its chance of achieving its goals. Some popular accounts use the term "artificial intelligence" to describe machines that mimic "cognitive" functions that humans associate with the human mind, such as "learning" and "problem solving", however this definition is rejected by major AI researchers. AI applications include advanced web search engines (i.e. Google), recommendation systems (used by YouTube, Amazon and Netflix), understanding human speech (such as Siri or Alexa), self-driving cars (e.g. Tesla), and competing at the highest level in strategic game systems (such as chess and Go), As machines become increasingly capable, tasks considered to require "intelligence" are often removed from the definition of AI, a phenomenon known as the AI effect. For instance, optical character recognition is frequently excluded from things considered to be AI, having become a routine technology.
"""

In [44]:
set_summary_length(ai_blog_wiki)

218
7


(3, 55, 72)

In [61]:
import torch
import sumy
import gensim
from gensim.summarization import summarize
import nltk
import re
nltk.download('punkt')
from transformers import BartForConditionalGeneration, BartTokenizer, BartConfig,BartModel
from transformers import XLMWithLMHeadModel, XLMTokenizer
from transformers import T5Tokenizer, T5Config, T5ForConditionalGeneration
from transformers import BigBirdPegasusForConditionalGeneration, AutoTokenizer
from transformers import PegasusForConditionalGeneration, PegasusTokenizer
from bs4 import BeautifulSoup 
# Import the LexRank summarizer
from sumy.summarizers.lex_rank import LexRankSummarizer
# Importing the parser and tokenizer
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
# Import the LexRank summarizer
from sumy.summarizers.lex_rank import LexRankSummarizer
lex_rank_summarizer = LexRankSummarizer() 
from sumy.summarizers.lsa import LsaSummarizer
lsa_summarizer=LsaSummarizer()
# Parsing the text string using PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.parsers.plaintext import PlaintextParser


class TextSummarizer:
  def __init__(self):
    # Instantiating the model and tokenizer bart
    self.tokenizer_bart=BartTokenizer.from_pretrained('facebook/bart-large-cnn')
    self.model_bart=BartForConditionalGeneration.from_pretrained('facebook/bart-large-cnn')
    # Instantiating the model and tokenizer t5
    self.model_t5 = T5ForConditionalGeneration.from_pretrained('t5-small')
    self.tokenizer_t5 = T5Tokenizer.from_pretrained('t5-small')
    # Instantiating the model and tokenizer google_bigbird
    self.model_BigBird = BigBirdPegasusForConditionalGeneration.from_pretrained("google/bigbird-pegasus-large-arxiv", attention_type="original_full")
    self.tokenizer_BigBird = AutoTokenizer.from_pretrained("google/bigbird-pegasus-large-arxiv")
    # by default encoder-attention is `block_sparse` with num_random_blocks=3, block_size=64
    
    self.tokenizer_Pegasus = PegasusTokenizer.from_pretrained('google/pegasus-xsum')
    self.model_Pegasus = PegasusForConditionalGeneration.from_pretrained('google/pegasus-xsum')


  def text_summarizer(self, text):
    min_len, max_len,num_sentences = set_summary_length(text)
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    text_summary_dict = {}

    text_summary_dict['_text_'] = text
    # Encoding the inputs and passing them to model.generate()
    inputs = self.tokenizer_bart.batch_encode_plus([text],return_tensors='pt',truncation=True)
    summary_ids = self.model_bart.generate(inputs['input_ids'], early_stopping=True,min_length = min_len, max_length=max_len)
    bart_summary = self.tokenizer_bart.decode(summary_ids[0], skip_special_tokens=True)
    # print(bart_summary)
    text_summary_dict['bart_summary'] = bart_summary

    inputs = self.tokenizer_BigBird(text, return_tensors='pt')
    summary_ids = self.model_BigBird.generate(**inputs,min_length=min_len, max_length=max_len)
    summary_BigBird = self.tokenizer_BigBird.batch_decode(summary_ids, skip_special_tokens=True)
    text_summary_dict['summary_BigBird'] = summary_BigBird
    print(summary_BigBird)

    # google pegasus summarization

    inputs = self.tokenizer_Pegasus([text], truncation=True, padding='longest', return_tensors="pt")
    summary_ids = self.model_Pegasus.generate(inputs['input_ids'], min_length=min_len, max_length=max_len)
    summary_google_pegasus = self.tokenizer_Pegasus.batch_decode(summary_ids, skip_special_tokens=True)
    text_summary_dict['summary_google_pegasus'] = summary_google_pegasus
    
    t5_text = "summarize:" + text
    # encoding the input text
    input_ids=self.tokenizer_t5.encode(t5_text, return_tensors='pt')
    summary_ids = self.model_t5.generate(input_ids,early_stopping=True, min_length = min_len, max_length=max_len)
    t5_summary = self.tokenizer_t5.decode(summary_ids[0], skip_special_tokens=True)
    text_summary_dict['t5_summary'] = t5_summary
    # gensim_summary
    summary_genensim = summarize(text)
    text_summary_dict['gensim_summary'] = summary_genensim
    my_parser = PlaintextParser.from_string(text,Tokenizer('english'))
    lexrank_summary_sentences = lex_rank_summarizer(my_parser.document,sentences_count=num_sentences)
    lexrank_summary = ""
    for sentence in lexrank_summary_sentences:
      lexrank_summary= lexrank_summary + " "+ str(sentence)
    text_summary_dict['lexrank_summary'] = lexrank_summary
    # creating the lsa summarizer
    parser=PlaintextParser.from_string(text,Tokenizer('english'))
    lsa_summary_sentences= lsa_summarizer(parser.document,num_sentences)
    lsa_summary =""
    #  lsa summary
    for sentence in lsa_summary_sentences:
        lsa_summary= lsa_summary+" "+ str(sentence)
    text_summary_dict['lsa_summary'] = lsa_summary
    return text_summary_dict

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [62]:
textSummarizer = TextSummarizer()

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.55k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.51G [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/1.82M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.36M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.12G [00:00<?, ?B/s]

In [64]:
text_summary_dict = textSummarizer.text_summarizer(ai_blog_wiki)

216
5
['the field of artificial intelligence ( i.e., natural intelligence ) has been rejected by major researchers due to a lack of applications. however, i.e., i.e., i.e., i.e., natural intelligence demonstrated by humans or machines ( as opposed to intelligence demonstrated by']


In [45]:
! pip install fpdf

  Created wheel for fpdf: filename=fpdf-1.7.2-py2.py3-none-any.whl size=40725 sha256=b9e5cdc97757e01488d9198e50bcf8ec2f0554f37a241ca326de8b647f6f6170
  Stored in directory: /root/.cache/pip/wheels/d7/ca/c8/86467e7957bbbcbdf4cf4870fc7dc95e9a16404b2e3c3a98c3
Successfully built fpdf


In [99]:
from transformers.models import auto
from fpdf import FPDF
# save FPDF() class into a  variable pdf
pdf = FPDF()
# pdf.set_auto_page_break(auto=True)
# Add a page
pdf.add_page()
  
# set style and size of font 
# that you want in the pdf
pdf.set_font("Arial", size = 11)

pdf.cell(200, 10, txt = "Text:",
         ln = 2, align = 'L')
pdf.multi_cell(200, 10, txt = ai_blog_wiki,
         align = 'L')
  
# create a cell
pdf.cell(200, 10, txt = "Summary",   ln = 1, align = 'C')

for summary_name in text_summary_dict:
  pdf.cell(200, 10, txt = summary_name,
         ln = 2, align = 'C')
  
  summary =text_summary_dict[summary_name]

  pdf.multi_cell(200, 10, txt = "".join(summary), align = 'L')
  
# add another cell

  
# save the pdf with name .pdf
pdf.output( "/content/summary_results.pdf",'F')

''